In [1]:
# Required imports.
from py import calc_ubi
from py.loss_functions import loss_metrics
from py.optimize import optimize

In [2]:
CATEGORIES = ['senior', 'child', 'dis_1', 'dis_2', 'dis_3',
              'NORTH_EAST', 'NORTH_WEST', 'YORKSHIRE', 'EAST_MIDLANDS',
              'WEST_MIDLANDS', 'EAST_OF_ENGLAND', 'LONDON', 'SOUTH_EAST',
              'SOUTH_WEST', 'WALES', 'SCOTLAND', 'NORTHERN_IRELAND']

# Define bounds
BASE_LIMITS = (40, 240)  # Child, working-age adult, senior.
PWD_LIMITS = (0, 40)  # Base, severe, and enhanced disability UBI supplements.
GEO_LIMITS = (-50, 50)  # Will be relative to a baseline geo.
# Skip adult which is calculated.
limits = [BASE_LIMITS] * 2 + [PWD_LIMITS] * 3 + [GEO_LIMITS] * 12

input_dict = {category : limit for category, limit in zip(CATEGORIES, limits)}
print (input_dict)

{'senior': (40, 240), 'child': (40, 240), 'dis_1': (0, 40), 'dis_2': (0, 40), 'dis_3': (0, 40), 'NORTH_EAST': (-50, 50), 'NORTH_WEST': (-50, 50), 'YORKSHIRE': (-50, 50), 'EAST_MIDLANDS': (-50, 50), 'WEST_MIDLANDS': (-50, 50), 'EAST_OF_ENGLAND': (-50, 50), 'LONDON': (-50, 50), 'SOUTH_EAST': (-50, 50), 'SOUTH_WEST': (-50, 50), 'WALES': (-50, 50), 'SCOTLAND': (-50, 50), 'NORTHERN_IRELAND': (-50, 50)}


In [ ]:
reform_1 = optimize(input_dict, 'mean_pct_loss', 'reform_1',
                    path="~/frs", verbose=False, maxiter=20, seed=0)

In [ ]:
print(reform_1)

In [ ]:
reform_2 = optimize(input_dict, 'mean_pct_loss', 'reform_2', 
                    path="~/frs", verbose=False, maxiter=20, seed=0)

In [ ]:
print(reform_2)

In [ ]:
reform_3 = optimize(input_dict, 'mean_pct_loss', 'reform_3', 
                    path="~/frs", verbose=False, maxiter=20, seed=0)

In [ ]:
print(reform_3)